# Run Molecular Dynamics Trajectories with Psi4
We want to see how closely our ML models adhere to trajectories produced without surrogates

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from fff.sampling.md import MolecularDynamics
from fff.simulation.utils import read_from_string
from ase.optimize import QuasiNewton
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.calculators.psi4 import Psi4
from ase.db import connect
from ase.io import write
from ttm.ase import TTMCalculator
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import warnings

/home/lward/miniconda3/envs/fastforce/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration

In [2]:
out_dir = Path('trajectories')

## Load in Example Structures
Use the same ones from our nwchem-evaluation

## Run Molecular Dynamics with DFT
Run a 1000 timesteps and save an entry every 10

In [3]:
psi4 = Psi4(method='pbe0', basis='aug-cc-pvdz', num_threads=12)

  Threads set to 12 by Python driver.


Test with diff

In [ ]:
out_dir.mkdir(exist_ok=True)
db_out_path = 'methane-relaxed-dft.db'
with connect('methane-added.db') as db:
    for row in tqdm(db.select(''), total=db.count()):
        # Check if this structure is done
        if db.count(filename=row.filename, state='relaxed') > 0: 
            continue

        # Read the structure and set its calculator
        atoms = row.toatoms()
        atoms.set_calculator(psi4)

        # Get the initial energy
        forces = atoms.get_forces()
        with connect(db_out_path) as db_out:
            db_out.write(atoms, state='unrelaxed', **row.key_value_pairs)


        # Relax it
        opt = QuasiNewton(atoms, logfile='opt.log')
        opt.run(fmax=0.02)
        with connect(db_out_path) as db_out:
            db_out.write(atoms, state='relaxed', **row.key_value_pairs)


  0%|                                                                                                                                                                                 | 0/128 [00:00<?, ?it/s]

  Threads set to 12 by Python driver.
